In [ ]:
import os
import requests
import numpy as np
import pyvista as pv
from datetime import datetime
from opendrive import OpenDrive

In [ ]:
path = os.path.dirname(os.path.curdir)
file = os.path.join(path, "./data", "Roundabout8Course.xodr")
# file = os.path.join(path, "./data", "Crossing8Course.xodr")
# file = os.path.join(path, "./data", "CrossingComplex8Course.xodr")
# file = os.path.join(path, "./data", "Town4.xodr")

# # To retrieve a file from the web
# uri = "https://raw.githubusercontent.com/carla-simulator/opendrive-test-files/master/OpenDrive/Town04.xodr"
# res = requests.get(uri)
# od = OpenDrive.from_string(res.text)

t_start = datetime.now()

od = OpenDrive.parse(file)

rm = od.get_road_mesh_new(s_step=1.0)
mk = od.get_roadmarker_mesh(s_step=1.0)

t_ela = (datetime.now() - t_start).microseconds / 1000

print(t_ela)

In [ ]:

class TriMesh():
    points: list[list[np.float64]]

    def __init__(self, inside, outside):
        # weld common points together
        comb = [ (i, o) for i,o in zip(inside, outside) if i is not o ]
        self.points = [ x for y in comb for x in y ]
        # this could be suboptimal if points are the same
        # self.points = [ x for y in zip(inside, outside) for x in y ]

    def get_points(self):
        return np.array(self.points)

    def get_strips(self):
        res = [len(self.points)]
        return res + [i for i in range(len(self.points))]

    def get_mesh(self):
        return pv.PolyData(self.points, strips=self.get_strips())

In [ ]:
pv.set_plot_theme("document")

pl = pv.Plotter()

lane_color = { "standard": "dimgray", "restricted": "darkred", "shoulder": "green", "driving": "dimgray", "sidewalk": "lightgray", "border": "gray" }

for road_id, road_data in rm.items():
    for section_id, section_data in road_data.items():
        lane_bounds = []

        for lane_id, lane_data in section_data.items():
            # print(lane_id)
            
            lane = lane_data['data']
             
            if lane.type != "none":
                trm = TriMesh(lane_data['in'], lane_data['out'])
                mesh = trm.get_mesh()  
                lclr = lane_color[lane.type]
                pl.add_mesh(mesh, show_edges=False, color=pv.Color(lclr, opacity=1.0))

                if int(lane.id) < -1:

                    trm = TriMesh(lane_data['in'], lane_bounds)
                    mesh = trm.get_mesh()
                    pl.add_mesh(mesh, show_edges=False, color=pv.Color(lclr, opacity=1.0))
              
                if int(lane.id) > 1:

                    trm = TriMesh(lane_data['in'], lane_bounds)
                    mesh = trm.get_mesh()
                    pl.add_mesh(mesh, show_edges=False, color=pv.Color(lclr, opacity=1.0))
                    
                lane_bounds = lane_data['out']
                


for road_id, road_data in mk.items():
    for section_id, section_data in road_data.items():
        for lane_id, lane_data in section_data.items():
            for marker_id, marker_data in lane_data.items():

                marker = marker_data['data']

                trm = TriMesh(marker_data['in'], marker_data['out'])
                mesh = trm.get_mesh()

                mkcol = "white" if marker.color == "standard" else marker.color
                if marker.type != "none":
                    pl.add_mesh(mesh, show_edges=False, color=pv.Color(mkcol, opacity=1.0))

pl.show(jupyter_backend='client')
